# Machine Temperature Dataset – Exploration

In this notebook, we conduct an exploratory analysis of the machine temperature dataset from the NAB (Numenta Anomaly Benchmark). The objective is to gain initial insights into the dataset structure and visualize patterns.

## Setup and Libraries
Importing necessary libraries and setting up configurations and data for the analysis.

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# load data
url = "https://raw.githubusercontent.com/hjalalin/nab-anomaly-detection/main/data/realKnownCause/machine_temperature_system_failure.csv"
df = pd.read_csv(url)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Load labeled anomaly windows from NAB
url = "https://raw.githubusercontent.com/numenta/NAB/master/labels/combined_windows.json"
response = requests.get(url)
labels = response.json()

# Extract windows for our dataset
label_key = "realKnownCause/machine_temperature_system_failure.csv"
anomaly_windows = [
    (pd.to_datetime(start), pd.to_datetime(end))
    for start, end in labels[label_key]
]

## Dataset Inspection
The following is an overview of the dataset to understand its basic structure, data types, and statistical characteristics.


In [ ]:
print("Shape:", df.shape)
print('')
print('Head:')
print(df.head())
print('')
print('statistical summary:')
print(df.describe())


## Plot Time Series
This plot shows the machine temperature over time. It visualizes trends patterns, sudden shifts or spikes that may indicate abnormal behavior or system faults.

In [ ]:
# Plot full series with anomaly windows shaded
plt.figure(figsize=(14, 4))
plt.plot(df.index, df['value'], label='Machine Temperature')

for idx, (start, end) in enumerate(anomaly_windows):
    plt.axvspan(start, end, color='red', alpha=0.3,
               label='Failure Time' if idx == 0 else "_nolegend_")

plt.title("Machine Temperature with NAB Labeled Anomalies")
plt.ylabel("Temperature (°C)")
plt.xlabel("Timestamp")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

The temperature readings are generally stable but occasionally exhibit sudden, significant drops. The red-shaded intervals specifically indicate known system failure periods identified by the NAB dataset. Although some of these abrupt temperature decreases align with labeled failures, some notable drops occur outside these intervals. Such unlabeled drops may represent other equipment shutdowns or sensor faults.

## Temperature Distribution
This section explores the distribution of temperature readings during both normal operation and known failure periods.

In [ ]:
anomaly_values = pd.concat([
    df[(df.index >= pd.Timestamp(start)) & (df.index <= pd.Timestamp(end))]
    for start, end in anomaly_windows
])
normal_values = df.drop(anomaly_values.index)

plt.figure(figsize=(14, 4))
sns.histplot(normal_values['value'], bins=50, color='steelblue', alpha=0.6, label='Normal', kde=False)
sns.histplot(anomaly_values['value'], bins=50, color='salmon', alpha=0.8, label='Failure Period', kde=False)

plt.xlabel('Temperature (°C)')
plt.ylabel('Count')
plt.title('Temperature Distribution with NAB-Labeled Failures')
plt.legend()
plt.grid(True)
plt.show()


Normal readings (blue) dominate the histogram, with most values clustered between 80°C and 100°C. The overall distribution of temperature readings in normal conditon is left-skewed, with a noticeable tail toward lower temperatures. Some of these low-temperature readings during normal operation may indicate equipment shutdowns, startup phases, or sensor artifacts, and do not necessarily indicate system faults.
Failure-period readings (salmon) are concentrated more heavily in the lower temperature range, reflecting sudden drops commonly associated with known system failures.
However, there is significant overlap between normal and failure-period temperature values, indicating that temperature alone is not a sufficient indicator of failure. While many failures coincide with low-temperature readings, similar drops can also occur under normal conditions. 